# Code implementation exploration
This Notebook has been created with the goal to explore the way the code implementation of stable-baselines and openai actually work, based on small "dummy"-cases.
   
   
1. Environment
2. Agent framework

## 1. Environment

In [1]:
import numpy as np
import pandas as pd
import gym
from gym import error, spaces, utils
from stable_baselines3.common.vec_env import DummyVecEnv

(config) no crisis measure selected.


In [2]:
from environment.FinancialMarketEnv import FinancialMarketEnv
from config.config import *
from setup_functions import *

In [3]:
# import dummy data
dumdat = pd.read_csv("data/preprocessed/old/done_data.csv", index_col=0)
dumdat = dumdat[["datadate", "tic", "adjcp"]]
dumdat.index = dumdat["datadate"].factorize()[0]
dumdat

,datadate,tic,adjcp
0,20090102,AAPL,12.964286
0,20090102,AXP,19.330000
0,20090102,BA,45.250000
0,20090102,CAT,46.910000
0,20090102,CSCO,16.960000
...,...,...,...
2925,20200817,V,199.430000
2925,20200817,VZ,58.780000
2925,20200817,WBA,41.175000
2925,20200817,WMT,135.600000


In [4]:
#dumdat = dumdat.loc[dumdat["tic"].isin(["AAPL", "AXP"])]
#dumdat.tic.unique()

In [5]:
# only pick 1 stock
dumdat = dumdat.loc[dumdat["tic"].isin(["AAPL"])]
# reduce data set to 5 days (4 possible steps)
#episode_length_steps = 5 
#dumdat = dumdat.loc[1-1:episode_length_steps-1]
#print("number of possible steps: ", len(dumdat.index.unique())-1)
dumdat.head(5)

,datadate,tic,adjcp
0,20090102,AAPL,12.964286
1,20090105,AAPL,13.511429
2,20090106,AAPL,13.288571
3,20090107,AAPL,13.001429
4,20090108,AAPL,13.242857


In [6]:
dumdat.tail(5)

,datadate,tic,adjcp
2921,20200811,AAPL,437.50
2922,20200812,AAPL,452.04
2923,20200813,AAPL,460.04
2924,20200814,AAPL,459.63
2925,20200817,AAPL,458.43


In [7]:
dumdat = dumdat.sort_values(["datadate", "tic"])
dumdat.index = dumdat["datadate"].factorize()[0]
dumdat

,datadate,tic,adjcp
0,20090102,AAPL,12.964286
1,20090105,AAPL,13.511429
2,20090106,AAPL,13.288571
3,20090107,AAPL,13.001429
4,20090108,AAPL,13.242857
...,...,...,...
2921,20200811,AAPL,437.500000
2922,20200812,AAPL,452.040000
2923,20200813,AAPL,460.040000
2924,20200814,AAPL,459.630000


In [8]:
dum_train = dumdat[dumdat.index<=2000]
dum_test = dumdat[dumdat.index>2000]

In [9]:
dum_train.tic.unique()

array(['AAPL'], dtype=object)

In [10]:
env_train = FinancialMarketEnv(df=dum_train, 
                               features_list=["adjcp"],
                               day=0,
                               iteration=1,  # only used for logging.infoing
                                                          model_name="ppo",
                                                          # only used for logging.info
                                                          mode="train",
                                                          crisis_measure=None,
                                                          crisis_threshold=None,
                                                          hmax_normalize=100,
                                                          initial_cash_balance=1000000,
                                                          transaction_fee_percent=0.001,
                                                          reward_scaling=1e-4,
                                                          assets_dim=1,
                                                          shape_observation_space=1+1+1,
                                                          initial=True, # for validation, we always have an "initial state"
                                                          previous_state=[],
                                                          previous_asset_price=None,
                                                          price_colname="adjcp",
                                                          results_dir="dummyresults",
                                                          reset_counter=0)

{'cash': [1000000], 'n_asset_holdings': [0], 'adjcp': [12.964285714285714]}
<class 'dict'>


In [11]:
vecenv_train = DummyVecEnv([lambda: FinancialMarketEnv(df=dum_train,  # was StockEnvTrain
                                                          features_list=["adjcp"],
                                                          day=0,
                                                          iteration=1,  # only used for logging.infoing
                                                          model_name="ppo",
                                                          # only used for logging.info
                                                          mode="train",
                                                          crisis_measure=None,
                                                          crisis_threshold=None,
                                                          hmax_normalize=100,
                                                          initial_cash_balance=1000000,
                                                          transaction_fee_percent=0.001,
                                                          reward_scaling=1e-4,
                                                          assets_dim=1,
                                                          shape_observation_space=1+1+1, # cash, price*stocks, n_holdings*stocks
                                                          initial=True, # for validation, we always have an "initial state"
                                                          previous_state=[],
                                                          previous_asset_price=None,
                                                          price_colname="adjcp",
                                                          results_dir="dummyresults",
                                                          reset_counter=0)])

{'cash': [1000000], 'n_asset_holdings': [0], 'adjcp': [12.964285714285714]}
<class 'dict'>
shapes:  {None: (3,)}
self.keys:  [None]


In [12]:
vecenv_train.reset()

[1.00000000e+06 0.00000000e+00 1.29642857e+01]
key:  None
obs:  [1.00000000e+06 0.00000000e+00 1.29642857e+01]
1. elem of obs:  1000000.0
type of 1. elem:  <class 'numpy.float64'>
len of obs:  3
type obs:  <class 'numpy.ndarray'>
env_ids:  0
buf_obs:  OrderedDict([(None, array([[0., 0., 0.]], dtype=float32))])
buf_obs[None]:  [[0. 0. 0.]]
buf_obs[None][env_idx]:  [0. 0. 0.]
a


array([[1.0000000e+06, 0.0000000e+00, 1.2964286e+01]], dtype=float32)

In [13]:
results_dir, trained_dir = create_dirs(mode="run_dir", dummydata=True)
print(results_dir, trained_dir)

dummyresults\results\06-23-2021_04-14-35_ppo__fm1_st dummyresults\trained_models\06-23-2021_04-14-35_ppo__rew_addPFVal_fm1_st


In [14]:
logsave_path = os.path.join(results_dir, "_LOGGINGS")
os.makedirs(logsave_path)
print(logsave_path)

dummyresults\results\06-23-2021_04-14-35_ppo__fm1_st\_LOGGINGS


In [15]:
config_logging_to_txt(results_dir, trained_dir, logsave_path=logsave_path)

In [16]:
logging.basicConfig(filename=os.path.join(logsave_path, "run_log"),
                    filemode='a',
                    format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                    # asctime = time, msecs, name = name of who runned it (?), levelname (e.g. DEBUG, INFO => verbosity), message
                    datefmt='%H:%M:%S',
                    #level=logging.INFO
                    #level=logging.DEBUG
                    level=logging.NOTSET)

In [17]:
env_train.seed(11)
env_train.action_space.seed(11)

[11]

In [18]:
model = PPO(policy='MlpPolicy',  # was PPO2 in sb2 tf1.x
            env=vecenv_train,  # environment where the agent learns and acts
            seed=11,

            ent_coef=agent_params._ppo.ENT_COEF,  # entropy coefficient # TODO: !!!
            # default parameters (unchanged from default as given by stable-baselines)
            learning_rate=agent_params._ppo.LEARNING_RATE,  # can also be variable, e.g. a function of the current progress remaining etc.
            # todo: where is learning rate used exactly?
            # nminibatches=8, # todo: was in old paper
            n_steps=agent_params._ppo.N_STEPS,
            n_epochs=agent_params._ppo.N_EPOCHS, #todo: surrogate loss?
            gamma=agent_params._ppo.GAMMA, # todo: would taking an actual interest rate make sense?
            gae_lambda=agent_params._ppo.GAE_LAMBDA, # todo: ?
            clip_range=agent_params._ppo.CLIP_RANGE,
            clip_range_vf=agent_params._ppo.CLIP_RANGE_VF, # todo: ?
            vf_coef=agent_params._ppo.VF_COEF,
            max_grad_norm=agent_params._ppo.MAX_GRAD_NORM,
            use_sde=agent_params._ppo.USE_SDE, # todo: ?
            sde_sample_freq=agent_params._ppo.SDE_SAMPLE_FREQ,
            target_kl=agent_params._ppo.TARGET_KL,
            tensorboard_log=agent_params._ppo.TENSORBOARD_LOG,
            create_eval_env=agent_params._ppo.CREATE_EVAL_ENV,
            policy_kwargs=agent_params._ppo.POLICY_KWARGS,
            verbose=agent_params._ppo.VERBOSE,
            device=agent_params._ppo.DEVICE,
            _init_setup_model=agent_params._ppo.INIT_SETUP_MODEL)

In [19]:
model.learn(total_timesteps= 100000)

[1.00000000e+06 0.00000000e+00 1.29642857e+01]
key:  None
obs:  [1.00000000e+06 0.00000000e+00 1.29642857e+01]
1. elem of obs:  1000000.0
type of 1. elem:  <class 'numpy.float64'>
len of obs:  3
type obs:  <class 'numpy.ndarray'>
env_ids:  0
buf_obs:  OrderedDict([(None, array([[1.0000000e+06, 0.0000000e+00, 1.2964286e+01]], dtype=float32))])
buf_obs[None]:  [[1.0000000e+06 0.0000000e+00 1.2964286e+01]]
buf_obs[None][env_idx]:  [1.0000000e+06 0.0000000e+00 1.2964286e+01]
a
key:  None
obs:  [9.98702275e+05 1.00000000e+02 1.35114286e+01]
1. elem of obs:  998702.275
type of 1. elem:  <class 'numpy.float64'>
len of obs:  3
type obs:  <class 'numpy.ndarray'>
env_ids:  0
buf_obs:  OrderedDict([(None, array([[1.0000000e+06, 0.0000000e+00, 1.2964286e+01]], dtype=float32))])
buf_obs[None]:  [[1.0000000e+06 0.0000000e+00 1.2964286e+01]]
buf_obs[None][env_idx]:  [1.0000000e+06 0.0000000e+00 1.2964286e+01]
a
key:  None
obs:  [9.98188327e+05 1.38000000e+02 1.32885714e+01]
1. elem of obs:  998188.32

key:  None
obs:  [9.94356060e+05 4.65000000e+02 1.41657143e+01]
1. elem of obs:  994356.0595785711
type of 1. elem:  <class 'numpy.float64'>
len of obs:  3
type obs:  <class 'numpy.ndarray'>
env_ids:  0
buf_obs:  OrderedDict([(None, array([[9.9312350e+05, 5.5200000e+02, 1.4181429e+01]], dtype=float32))])
buf_obs[None]:  [[9.9312350e+05 5.5200000e+02 1.4181429e+01]]
buf_obs[None][env_idx]:  [9.9312350e+05 5.5200000e+02 1.4181429e+01]
a
key:  None
obs:  [9.95771214e+05 3.65000000e+02 1.35042857e+01]
1. elem of obs:  995771.214435714
type of 1. elem:  <class 'numpy.float64'>
len of obs:  3
type obs:  <class 'numpy.ndarray'>
env_ids:  0
buf_obs:  OrderedDict([(None, array([[9.9435606e+05, 4.6500000e+02, 1.4165714e+01]], dtype=float32))])
buf_obs[None]:  [[9.9435606e+05 4.6500000e+02 1.4165714e+01]]
buf_obs[None][env_idx]:  [9.9435606e+05 4.6500000e+02 1.4165714e+01]
a
key:  None
obs:  [9.95487341e+05 3.86000000e+02 1.34814286e+01]
1. elem of obs:  995487.340845714
type of 1. elem:  <class 

buf_obs[None]:  [[9.9809519e+05 1.5900000e+02 1.6921429e+01]]
buf_obs[None][env_idx]:  [9.9809519e+05 1.5900000e+02 1.6921429e+01]
a
key:  None
obs:  [9.97910347e+05 1.72000000e+02 1.66171429e+01]
1. elem of obs:  997910.3473069282
type of 1. elem:  <class 'numpy.float64'>
len of obs:  3
type obs:  <class 'numpy.ndarray'>
env_ids:  0
buf_obs:  OrderedDict([(None, array([[9.9916019e+05, 9.6000000e+01, 1.6428572e+01]], dtype=float32))])
buf_obs[None]:  [[9.9916019e+05 9.6000000e+01 1.6428572e+01]]
buf_obs[None][env_idx]:  [9.9916019e+05 9.6000000e+01 1.6428572e+01]
a
key:  None
obs:  [9.99570400e+05 7.20000000e+01 1.70814286e+01]
1. elem of obs:  999570.3998783568
type of 1. elem:  <class 'numpy.float64'>
len of obs:  3
type obs:  <class 'numpy.ndarray'>
env_ids:  0
buf_obs:  OrderedDict([(None, array([[9.9791038e+05, 1.7200000e+02, 1.6617144e+01]], dtype=float32))])
buf_obs[None]:  [[9.9791038e+05 1.7200000e+02 1.6617144e+01]]
buf_obs[None][env_idx]:  [9.9791038e+05 1.7200000e+02 1.6617

 None
obs:  [9.97195922e+05 2.00000000e+02 1.89585714e+01]
1. elem of obs:  997195.9217083569
type of 1. elem:  <class 'numpy.float64'>
len of obs:  3
type obs:  <class 'numpy.ndarray'>
env_ids:  0
buf_obs:  OrderedDict([(None, array([[9.9908450e+05, 1.0000000e+02, 1.8867144e+01]], dtype=float32))])
buf_obs[None]:  [[9.9908450e+05 1.0000000e+02 1.8867144e+01]]
buf_obs[None][env_idx]:  [9.9908450e+05 1.0000000e+02 1.8867144e+01]
a
key:  None
obs:  [9.95298169e+05 3.00000000e+02 1.89285714e+01]
1. elem of obs:  995298.1687083569
type of 1. elem:  <class 'numpy.float64'>
len of obs:  3
type obs:  <class 'numpy.ndarray'>
env_ids:  0
buf_obs:  OrderedDict([(None, array([[9.9719594e+05, 2.0000000e+02, 1.8958570e+01]], dtype=float32))])
buf_obs[None]:  [[9.9719594e+05 2.0000000e+02 1.8958570e+01]]
buf_obs[None][env_idx]:  [9.9719594e+05 2.0000000e+02 1.8958570e+01]
a
key:  None
obs:  [9.93403419e+05 4.00000000e+02 1.84371429e+01]
1. elem of obs:  993403.4187083569
type of 1. elem:  <class 'nu

key:  None
obs:  [9.95800218e+05 2.37000000e+02 1.91442857e+01]
1. elem of obs:  995800.2179569281
type of 1. elem:  <class 'numpy.float64'>
len of obs:  3
type obs:  <class 'numpy.ndarray'>
env_ids:  0
buf_obs:  OrderedDict([(None, array([[9.9546694e+05, 2.5400000e+02, 1.9624287e+01]], dtype=float32))])
buf_obs[None]:  [[9.9546694e+05 2.5400000e+02 1.9624287e+01]]
buf_obs[None][env_idx]:  [9.9546694e+05 2.5400000e+02 1.9624287e+01]
a
key:  None
obs:  [9.97712732e+05 1.37000000e+02 1.94600000e+01]
1. elem of obs:  997712.7320997852
type of 1. elem:  <class 'numpy.float64'>
len of obs:  3
type obs:  <class 'numpy.ndarray'>
env_ids:  0
buf_obs:  OrderedDict([(None, array([[9.9580019e+05, 2.3700000e+02, 1.9144285e+01]], dtype=float32))])
buf_obs[None]:  [[9.9580019e+05 2.3700000e+02 1.9144285e+01]]
buf_obs[None][env_idx]:  [9.9580019e+05 2.3700000e+02 1.9144285e+01]
a
key:  None
obs:  [9.99656786e+05 3.70000000e+01 1.99800000e+01]
1. elem of obs:  999656.7860997852
type of 1. elem:  <clas

...

obs:  [9.20999818e+05 1.90300000e+03 1.09360000e+02]
1. elem of obs:  920999.818097685
type of 1. elem:  <class 'numpy.float64'>
len of obs:  3
type obs:  <class 'numpy.ndarray'>
env_ids:  0
buf_obs:  OrderedDict([(None, array([[9.1010275e+05, 2.0030000e+03, 1.0908000e+02]], dtype=float32))])
buf_obs[None]:  [[9.1010275e+05 2.0030000e+03 1.0908000e+02]]
buf_obs[None][env_idx]:  [9.1010275e+05 2.0030000e+03 1.0908000e+02]
a
key:  None
obs:  [9.26243849e+05 1.85500000e+03 1.08510000e+02]
1. elem of obs:  926243.848817685
type of 1. elem:  <class 'numpy.float64'>
len of obs:  3
type obs:  <class 'numpy.ndarray'>
env_ids:  0
buf_obs:  OrderedDict([(None, array([[9.209998e+05, 1.903000e+03, 1.093600e+02]], dtype=float32))])
buf_obs[None]:  [[9.209998e+05 1.903000e+03 1.093600e+02]]
buf_obs[None][env_idx]:  [9.209998e+05 1.903000e+03 1.093600e+02]
a
key:  None
obs:  [9.15381998e+05 1.95500000e+03 1.08850000e+02]
1. elem of obs:  915381.997817685
type of 1. elem:  <class 'numpy.float64'>
len 

key:  None
obs:  [9.62780065e+05 1.50400000e+03 1.12180000e+02]
1. elem of obs:  962780.065367685
type of 1. elem:  <class 'numpy.float64'>
len of obs:  3
type obs:  <class 'numpy.ndarray'>
env_ids:  0
buf_obs:  OrderedDict([(None, array([[9.5606375e+05, 1.5630000e+03, 1.1395000e+02]], dtype=float32))])
buf_obs[None]:  [[9.5606375e+05 1.5630000e+03 1.1395000e+02]]
buf_obs[None][env_idx]:  [9.5606375e+05 1.5630000e+03 1.1395000e+02]
a
key:  None
obs:  [9.73986847e+05 1.40400000e+03 1.13050000e+02]
1. elem of obs:  973986.847367685
type of 1. elem:  <class 'numpy.float64'>
len of obs:  3
type obs:  <class 'numpy.ndarray'>
env_ids:  0
buf_obs:  OrderedDict([(None, array([[9.6278006e+05, 1.5040000e+03, 1.1218000e+02]], dtype=float32))])
buf_obs[None]:  [[9.6278006e+05 1.5040000e+03 1.1218000e+02]]
buf_obs[None][env_idx]:  [9.6278006e+05 1.5040000e+03 1.1218000e+02]
a
key:  None
obs:  [9.83699425e+05 1.31800000e+03 1.12520000e+02]
1. elem of obs:  983699.4250676851
type of 1. elem:  <class 

key:  None
obs:  [1.015951e+06 1.029000e+03 1.098300e+02]
1. elem of obs:  1015951.004197685
type of 1. elem:  <class 'numpy.float64'>
len of obs:  3
type obs:  <class 'numpy.ndarray'>
env_ids:  0
buf_obs:  OrderedDict([(None, array([[1.00703275e+06, 1.10900000e+03, 1.11589996e+02]], dtype=float32))])
buf_obs[None]:  [[1.00703275e+06 1.10900000e+03 1.11589996e+02]]
buf_obs[None][env_idx]:  [1.00703275e+06 1.10900000e+03 1.11589996e+02]
a
key:  None
obs:  [1.01858429e+06 1.00500000e+03 1.08840000e+02]
1. elem of obs:  1018584.2882776851
type of 1. elem:  <class 'numpy.float64'>
len of obs:  3
type obs:  <class 'numpy.ndarray'>
env_ids:  0
buf_obs:  OrderedDict([(None, array([[1.015951e+06, 1.029000e+03, 1.098300e+02]], dtype=float32))])
buf_obs[None]:  [[1.015951e+06 1.029000e+03 1.098300e+02]]
buf_obs[None][env_idx]:  [1.015951e+06 1.029000e+03 1.098300e+02]
a
key:  None
obs:  [1.01488003e+06 1.03900000e+03 1.10410000e+02]
1. elem of obs:  1014880.0277176851
type of 1. elem:  <class 'n


buf_obs:  OrderedDict([(None, array([[1.0137912e+06, 1.0530000e+03, 1.1173000e+02]], dtype=float32))])
buf_obs[None]:  [[1.0137912e+06 1.0530000e+03 1.1173000e+02]]
buf_obs[None][env_idx]:  [1.0137912e+06 1.0530000e+03 1.1173000e+02]
a
key:  None
obs:  [1.0123108e+06 1.0660000e+03 1.1123000e+02]
1. elem of obs:  1012310.8021576849
type of 1. elem:  <class 'numpy.float64'>
len of obs:  3
type obs:  <class 'numpy.ndarray'>
env_ids:  0
buf_obs:  OrderedDict([(None, array([[1.023502e+06, 9.660000e+02, 1.118000e+02]], dtype=float32))])
buf_obs[None]:  [[1.023502e+06 9.660000e+02 1.118000e+02]]
buf_obs[None][env_idx]:  [1.023502e+06 9.660000e+02 1.118000e+02]
a
key:  None
obs:  [1.02053359e+06 9.92000000e+02 1.11790000e+02]
1. elem of obs:  1020533.5911376849
type of 1. elem:  <class 'numpy.float64'>
len of obs:  3
type obs:  <class 'numpy.ndarray'>
env_ids:  0
buf_obs:  OrderedDict([(None, array([[1.0123108e+06, 1.0660000e+03, 1.1123000e+02]], dtype=float32))])
buf_obs[None]:  [[1.0123108e

FileNotFoundError: [Errno 2] No such file or directory: 'dummyresults\\datadates\\datadates_train_ppo_ep1_totalSteps_2000_finalStateCounter_1.csv'